# Tugas Rekayasa Data - Implementasi Data Cube on OULAD
## Kelompok 26

#### Mengimpor Library Atoti, Pandas, dan Numpy

In [96]:
import atoti as tt
import pandas as pd
import numpy as np

#### Mengimpor file csv dari tabel yang digunakan

In [97]:
df_Ass = pd.read_csv('assessments.csv')
df_stAss = pd.read_csv('studentAssessment.csv')
df_stInfo = pd.read_csv('studentInfo.csv')

## Preparasi Data

### Tabel Student Info

In [98]:
df_stInfo['id_student'] = df_stInfo['id_student'].astype('string')
df_stInfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32593 entries, 0 to 32592
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   code_module           32593 non-null  object
 1   code_presentation     32593 non-null  object
 2   id_student            32593 non-null  string
 3   gender                32593 non-null  object
 4   region                32593 non-null  object
 5   highest_education     32593 non-null  object
 6   imd_band              31482 non-null  object
 7   age_band              32593 non-null  object
 8   num_of_prev_attempts  32593 non-null  int64 
 9   studied_credits       32593 non-null  int64 
 10  disability            32593 non-null  object
 11  final_result          32593 non-null  object
dtypes: int64(2), object(9), string(1)
memory usage: 3.0+ MB


### Tabel Student Assesment

In [99]:
col = ['id_student','id_assessment']
df_stAss[col] = df_stAss[col].astype('string')
df_stAss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173912 entries, 0 to 173911
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id_assessment   173912 non-null  string 
 1   id_student      173912 non-null  string 
 2   date_submitted  173912 non-null  int64  
 3   is_banked       173912 non-null  int64  
 4   score           173739 non-null  float64
dtypes: float64(1), int64(2), string(2)
memory usage: 6.6 MB


### Tabel Assesment

In [100]:
df_Ass['id_assessment'] = df_Ass['id_assessment'].astype('string')
df_Ass.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   code_module        206 non-null    object 
 1   code_presentation  206 non-null    object 
 2   id_assessment      206 non-null    string 
 3   assessment_type    206 non-null    object 
 4   date               195 non-null    float64
 5   weight             206 non-null    float64
dtypes: float64(2), object(3), string(1)
memory usage: 9.8+ KB


## Mengecek nilai null pada tabel

### Nilai null pada Tabel Assesment

In [101]:
df_Ass.isnull().sum()

code_module           0
code_presentation     0
id_assessment         0
assessment_type       0
date                 11
weight                0
dtype: int64

Nilai null pada attribute date tidak masalah karena dalam mencari insight tidak menggunakan attribute tersebut.

In [11]:
df_Ass.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0


### Nilai null pada Tabel Student Assesment

In [102]:
df_stAss.isnull().sum()

id_assessment       0
id_student          0
date_submitted      0
is_banked           0
score             173
dtype: int64

Nilai null pada attribute score di sini dapat dianggap bahwa mahasiswa tersebut tidak mengumpulkan sehingga tidak mendapatkan nilai, maka nilai null di sini akan kami anggap sebagai nilai 0. Kemudian kami membuat rentang untuk mengumpulkan range nilai menjadi 4 kategori, supaya lebih mudah dipahami.

In [32]:
range = []
for row in df_stAss['score']:
    if row > 75.0 : range.append('75 < ')
    elif row > 50.0 : range.append('50-75')
    elif row > 25.0 : range.append('25-50')
    elif row > 0 or np.nan: range.append('25 > ')

In [107]:
df_stAss['range score'] = range

### Nilai null pada Tabel Student Info

In [62]:
df_stInfo.isnull().sum()

code_module                0
code_presentation          0
id_student                 0
gender                     0
region                     0
highest_education          0
imd_band                1111
age_band                   0
num_of_prev_attempts       0
studied_credits            0
disability                 0
final_result               0
dtype: int64

Nilai null pada attribute imd_band tidak masalah karena dalam mencari insight tidak menggunakan attribute tersebut

## Menampilkan Data Tabel yang Digunakan

In [109]:
ses = tt.Session()

Deleting existing "Unnamed" session to create the new one.


In [110]:
Ass = ses.read_pandas(df_Ass, keys=["id_assessment","code_module"], table_name="Assessments")
Ass.head()

code_presentation assessment_type   date  weight
code_module id_assessment                                                 
AAA         1752                      2013J             TMA   19.0    10.0
            1754                      2013J             TMA  117.0    20.0
            1756                      2013J             TMA  215.0    30.0
            1759                      2014J             TMA   54.0    20.0
            1761                      2014J             TMA  166.0    20.0

In [111]:
stInfo = ses.read_pandas(df_stInfo, keys=["id_student","code_module","final_result"], table_name="Student Info")
stInfo.head()

code_presentation gender  \
code_module id_student final_result                            
AAA         11391      Pass                     2013J      M   
            28400      Pass                     2013J      F   
            31604      Pass                     2013J      F   
            38053      Pass                     2013J      M   
            45462      Pass                     2013J      M   

                                                  region  \
code_module id_student final_result                        
AAA         11391      Pass          East Anglian Region   
            28400      Pass                     Scotland   
            31604      Pass            South East Region   
            38053      Pass                        Wales   
            45462      Pass                     Scotland   

                                         highest_education imd_band age_band  \
code_module id_student final_result                                            
AAA         11391      Pass               HE Qualification  90-100%     55<=   
            28400      Pass               HE Qualification   20-30%    35-55   
            31604      Pass          A Level or Equivalent   50-60%    35-55   
            38053      Pass          A Level or Equivalent   80-90%    35-55   
            45462      Pass               HE Qualification   30-40%     0-35   

                                     num_of_prev_attempts  studied_credits  \
code_module id_student final_result                                          
AAA         11391      Pass                             0              240   
            28400      Pass                             0               60   
            31604      Pass                             0               60   
            38053      Pass                             0               60   
            45462      Pass                             0               60   

                                    disability  
code_module id_student final_result             
AAA         11391      Pass                  N  
            28400      Pass                  N  
            31604      Pass                  N  
            38053      Pass                  N  
            45462      Pass                  N

In [112]:
stAss = ses.read_pandas(df_stAss, keys=["id_student","id_assessment","range score"], table_name="Student Assessments")
stAss.head()

date_submitted  is_banked  score
id_assessment id_student range score                                  
1752          11391      75 <                     18          0   78.0
              31604      50-75                    17          0   72.0
              63400      75 <                     19          0   83.0
              71361      75 <                     19          0   82.0
              100893     50-75                    17          0   63.0

## Membuat Data Cube

Menggabungkan 3 tabel yaitu Student Assessment, Assessment, dan Student Info untuk mendapatkan insight dari relasi tabel tersebut. Menggunakan metode join.

In [114]:
stAss.join(Ass, mapping={'id_assessment': 'id_assessment'})
stInfo.join(stAss, mapping={'id_student': 'id_student'})
cube1 = ses.create_cube(stInfo)

h, l, m = cube1.hierarchies, cube1.levels, cube1.measures

In [115]:
stInfo.head()

code_presentation gender  \
code_module id_student final_result                            
AAA         11391      Pass                     2013J      M   
            28400      Pass                     2013J      F   
            45462      Pass                     2013J      M   
            63400      Pass                     2013J      M   
            70464      Pass                     2013J      F   

                                                   region  \
code_module id_student final_result                         
AAA         11391      Pass           East Anglian Region   
            28400      Pass                      Scotland   
            45462      Pass                      Scotland   
            63400      Pass                      Scotland   
            70464      Pass          West Midlands Region   

                                         highest_education imd_band age_band  \
code_module id_student final_result                                            
AAA         11391      Pass               HE Qualification  90-100%     55<=   
            28400      Pass               HE Qualification   20-30%    35-55   
            45462      Pass               HE Qualification   30-40%     0-35   
            63400      Pass             Lower Than A Level   40-50%    35-55   
            70464      Pass          A Level or Equivalent   60-70%    35-55   

                                     num_of_prev_attempts  studied_credits  \
code_module id_student final_result                                          
AAA         11391      Pass                             0              240   
            28400      Pass                             0               60   
            45462      Pass                             0               60   
            63400      Pass                             0               60   
            70464      Pass                             0               60   

                                    disability  
code_module id_student final_result             
AAA         11391      Pass                  N  
            28400      Pass                  N  
            45462      Pass                  N  
            63400      Pass                  N  
            70464      Pass                  N

In [116]:
cube1

Cube(_name='Student Info', _hierarchies=Hierarchies(_create_hierarchy_from_arguments=<bound method Cube._create_hierarchy_from_arguments of ...>, _cube_name='Student Info'), _measures=Measures())

In [117]:
h

Hierarchies(_create_hierarchy_from_arguments=<bound method Cube._create_hierarchy_from_arguments of Cube(_name='Student Info', _hierarchies=..., _measures=Measures())>, _cube_name='Student Info')

In [71]:
l

Levels()

In [118]:
m

Measures()

## Membuat Visualisasi Insight

### Insight 1. Tren tingkat kelulusan berdasarkan tingkat pendidikan terakhir

In [119]:
# Visualisasi dalam bentuk pivot tabel
ses.visualize()

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

In [120]:
# Visualisasi dalam bentuk donut chart
ses.visualize()

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

In [121]:
# Dice untuk melihat jumlah berdasarkan tingkat pendidikan
cube1.query(
    m["contributors.COUNT"],
    levels=[l["highest_education"]]
)

,contributors.COUNT
highest_education,
A Level or Equivalent,"13,831"
HE Qualification,"4,648"
Lower Than A Level,"12,924"
No Formal quals,337
Post Graduate Qualification,304


In [122]:
# Slice data pada tingkat pendidikan "A Level or Equivalent"
cube1.query(
    m["contributors.COUNT"],
    filter=l["highest_education"] == "A Level or Equivalent",
)

,contributors.COUNT
0,"13,831"


In [123]:
# Dice pada 2 attribute yaitu tingkat pendidikan dan gender
cube1.query(m["contributors.COUNT"], levels=[l["highest_education"], l["gender"]])

contributors.COUNT
highest_education           gender                   
A Level or Equivalent       F                   6,275
                            M                   7,556
HE Qualification            F                   1,768
                            M                   2,880
Lower Than A Level          F                   6,169
                            M                   6,755
No Formal quals             F                     201
                            M                     136
Post Graduate Qualification F                      87
                            M                     217

### Insight 2. Tren persebaran gender berdasarkan tingkat pendidikan terakhir

In [124]:
# Visualisasi dalam bentuk pivot tabel
ses.visualize()

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

In [125]:
# Visualisasi dalam bentuk bar chart
ses.visualize()

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

### Insight 3. Pengaruh tipe assesment terhadap skor yang didapat

In [126]:
# Visualisasi dalam bentuk pivot tabel
ses.visualize()

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

In [127]:
# Visualisasi dalam bentuk stacked bars
ses.visualize()

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.

In [128]:
# Menambahkan attribute assessment_type untuk filtering
ses.visualize()

Open the notebook in JupyterLab with the atoti extension enabled to build this widget.